In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
import pyarrow.parquet as pq

In [2]:
dir_name = 'C:\Cloud\OneDrive - Emory University\Papers\PASC Cardiometabolic Risk Factors'
outcome_df = pd.read_parquet(dir_name + '\working\models\pcra201_ipw for cohort membership data.parquet' )
censoring_df = pd.read_parquet(dir_name + '\working\models\pcra202_ipw for outcome availability dataset.parquet' )
# censoring_df.drop(columns = ['matchid'],inplace = True)
# https://stackoverflow.com/questions/41815079/pandas-merge-join-two-data-frames-on-multiple-columns
censoring_df = censoring_df.merge(outcome_df,how = 'left',on = ['ID','COHORT'])
del outcome_df
censoring_df.head()

,ID,COHORT,in_index_date,in_bmi_ID,in_sbp_ID,in_ldl_ID,female,nhwhite,nhblack,hispanic,...,LOINC_48159_8_gtQ3,LOINC_27297_1_gtQ3,LOINC_50553_7_gtQ3,LOINC_53290_3_gtQ3,LOINC_49524_2_gtQ3,LOINC_29908_1_gtQ3,LOINC_49521_8_gtQ3,LOINC_62423_9_gtQ3,LOINC_24027_5_gtQ3,LOINC_11557_6_gtQ3
0,12MAR202320220187400000001,unexposed,1,1,0,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12MAR202320220187400000002,unexposed,1,1,1,0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12MAR202320220187400000003,historical,1,1,1,0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12MAR202320220187400000004,unexposed,1,1,0,1,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12MAR202320220187400000005,unexposed,1,1,1,0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
censoring_df.shape

(391009, 2987)

In [5]:
censoring_df = pd.get_dummies(censoring_df,prefix='',prefix_sep='_',
                                           columns=['site','calendar_month','payer_type_primary','payer_type_secondary'],drop_first=True)
censoring_df.shape

(391009, 3012)

# Code for Random Forest Classifier

In [18]:
rf_bmi = RandomForestClassifier(random_state=1,min_samples_leaf=10,n_estimators=1000)
rf_sbp = RandomForestClassifier(random_state=1,min_samples_leaf=10,n_estimators=1000)
rf_ldl = RandomForestClassifier(random_state=1,min_samples_leaf=10,n_estimators=1000)
# TRIAL 
# rf_bmi = RandomForestClassifier(random_state=1,min_samples_leaf=1000,n_estimators=10)

# Creating outcome variables

In [11]:
y_bmi = censoring_df['in_bmi_ID']
y_sbp = censoring_df['in_sbp_ID']
y_ldl = censoring_df['in_ldl_ID']
X = censoring_df.drop(['in_bmi_ID','in_sbp_ID','in_ldl_ID','COHORT','ID','matchid','index_date'], axis=1)


## Missing BMI

In [19]:
rf_bmi.fit(X,y_bmi)

RandomForestClassifier(min_samples_leaf=1000, n_estimators=10, random_state=1)

In [20]:
y_bmi_pred = rf_bmi.predict(X)
y_bmi_pred_proba = rf_bmi.predict_proba(X)
pd.DataFrame(y_bmi_pred_proba,columns=['absent','present']).to_csv(dir_name + '\working\models\pcra202_predicted probability for in_bmi_ID_min10_ntree1000.csv')
pd.crosstab(y_bmi,y_bmi_pred)

col_0,0,1
in_bmi_ID,,
0,60883,89443
1,25359,215324


## Missing SBP

In [ ]:
rf_sbp.fit(X,y_sbp)

In [ ]:
y_sbp_pred = rf_sbp.predict(X)
y_sbp_pred_proba = rf_sbp.predict_proba(X)
pd.DataFrame(y_sbp_pred_proba,columns=['absent','present']).to_csv(dir_name + '\working\models\pcra202_predicted probability for in_sbp_ID_min10_ntree1000.csv')
pd.crosstab(y_sbp,y_sbp_pred)

## Missing LDL

In [ ]:
rf_ldl.fit(X,y_ldl)

In [ ]:
y_ldl_pred = rf_ldl.predict(X)
y_ldl_pred_proba = rf_ldl.predict_proba(X)
pd.DataFrame(y_ldl_pred_proba,columns=['absent','present']).to_csv(dir_name + '\working\models\pcra202_predicted probability for in_ldl_ID_min10_ntree1000.csv')
pd.crosstab(y_ldl,y_ldl_pred)